In [12]:
import torch
import numpy as np
import slam
import nerf
import matplotlib.pyplot as plt
from importlib import reload
from torch.utils.data import DataLoader
import pandas as pd
torch.set_printoptions(linewidth=1000)

In [13]:
model, matching = slam.get_models()

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [14]:
reload(slam)
data = slam.images_to_nerf_inputs(
    image_prefix="indoor2", 
    model=model,
    matching=matching,
    force_reload=True)
data.shape
data[:10, :]

tensor([[ 0.0000,  0.0000,  0.0000, -0.3569, -0.3569,  0.8633,  0.5882,  0.5725,  0.5216],
        [ 0.0000,  0.0000,  0.0000, -0.3557, -0.3571,  0.8637,  0.5333,  0.5216,  0.4667],
        [ 0.0000,  0.0000,  0.0000, -0.3545, -0.3572,  0.8641,  0.4863,  0.4588,  0.3843],
        [ 0.0000,  0.0000,  0.0000, -0.3533, -0.3574,  0.8646,  0.8157,  0.7765,  0.5804],
        [ 0.0000,  0.0000,  0.0000, -0.3521, -0.3576,  0.8650,  0.8157,  0.7882,  0.5765],
        [ 0.0000,  0.0000,  0.0000, -0.3509, -0.3578,  0.8654,  0.5333,  0.5294,  0.4627],
        [ 0.0000,  0.0000,  0.0000, -0.3496, -0.3579,  0.8658,  0.4588,  0.4588,  0.4235],
        [ 0.0000,  0.0000,  0.0000, -0.3484, -0.3581,  0.8662,  0.4784,  0.4667,  0.4353],
        [ 0.0000,  0.0000,  0.0000, -0.3472, -0.3583,  0.8666,  0.4588,  0.4510,  0.4235],
        [ 0.0000,  0.0000,  0.0000, -0.3460, -0.3585,  0.8671,  0.4588,  0.4510,  0.4196]])

In [15]:
data_loader = DataLoader(data, batch_size=2**14, shuffle=True)

In [54]:
reload(nerf)
positions, directions, target_rgb = nerf.preprocess_data(data, 64, 1.0, 4.5)
nerf_model = nerf.NeRFModel()
nerf_model.eval()

NeRFModel(
  (pts_linears): ModuleList(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1-3): 3 x Linear(in_features=256, out_features=256, bias=True)
    (4): Linear(in_features=259, out_features=256, bias=True)
    (5-7): 3 x Linear(in_features=256, out_features=256, bias=True)
  )
  (views_linears): ModuleList(
    (0): Linear(in_features=259, out_features=128, bias=True)
  )
  (feature_linear): Linear(in_features=256, out_features=256, bias=True)
  (alpha_linear): Linear(in_features=256, out_features=1, bias=True)
  (rgb_linear): Linear(in_features=128, out_features=3, bias=True)
)

In [55]:
nerf.train(model=nerf_model, data_loader=data_loader)

input_pts shape: torch.Size([1048576, 3])
input_dirs shape: torch.Size([1048576, 3])
h shape after layer 0: torch.Size([1048576, 256])
h shape after layer 1: torch.Size([1048576, 256])
h shape after layer 2: torch.Size([1048576, 256])
h shape after layer 3: torch.Size([1048576, 256])
h shape after skip connection at layer 4: torch.Size([1048576, 259])
h shape after layer 4: torch.Size([1048576, 256])
h shape after layer 5: torch.Size([1048576, 256])
h shape after layer 6: torch.Size([1048576, 256])
h shape after layer 7: torch.Size([1048576, 256])
h shape after concatenating feature and input_dirs: torch.Size([1048576, 259])


RuntimeError: shape '[16384, 9]' is invalid for input of size 3145728